In [ ]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from typing import List, Union, Optional, Callable
import pickle

from ete3 import Tree, TreeNode
from gctree import CollapsedTree

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap
from GCTree_preparation import *
import warnings
warnings.filterwarnings("ignore")

path_to_storage = "/media/hieunguyen/HNSD01/storage/all_BSimons_datasets"

PROJECT = "220701_etc_biopsies"

output_type = "mouse_based_output"

path_to_trees = os.path.join(path_to_storage, PROJECT, "GCtrees/v0.2", output_type)

all_tree_folder = [item for item in pathlib.Path(path_to_trees).glob("*") if 
                   os.path.isfile(f"{str(item)}/02_dnapars/gctree.out.inference.1.nk") == True]

all_nk_files = [item for item in pathlib.Path(path_to_trees).glob("*/*/*gctree.out.inference.1.nk")]  
print(f"Number of trees: {len(all_tree_folder)}")   

/home/hieunguyen/miniconda3/envs/ete3_py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of trees: 7618


In [11]:
# loop throught all trees
treedir = all_tree_folder[0]

cloneid = treedir.name
treedir = str(treedir)

mouseID = cloneid.split("_")[0]
V_gene = cloneid.split("_")[1]
J_gene = cloneid.split("_")[2]
CDR3_len = cloneid.split("_")[3]

nk_path = f"{treedir}/02_dnapars/gctree.out.inference.1.nk"
ab_dict_path = os.path.join(treedir, "01_deduplicate", f"{cloneid}.abundance.csv")
abund_df = pd.read_csv(ab_dict_path, index_col=0, names=['val'])
ab_dict = abund_df.to_dict().get('val')
tree_path = treedir
tree = Tree(newick=nk_path, format=1)
if ab_dict is not None:
    for node in tree.traverse():
        node.add_feature('abundance', ab_dict.get(node.name, 0))
treeobj = GCtree(tree = tree, path = tree_path)

num_nodes = len(treeobj.nodes)
num_leaves = len(treeobj.leaves)
num_internal_nodes = len(treeobj.internal_nodes)
num_passthrough_nodes = len(treeobj.passthrough_nodes)
num_split_nodes = len(treeobj.split_nodes)
num_observed_nodes = len(treeobj.observed_nodes)
num_inferred_nodes = len(treeobj.inferred_nodes)

tmpdf = pd.DataFrame(
    {
        "cloneid": cloneid,
        "mouseID": mouseID,
        "V_gene": V_gene,
        "J_gene": J_gene,
        "CDR3_len": CDR3_len,
        "num_nodes": num_nodes,
        "num_leaves": num_leaves,
        "num_internal_nodes": num_internal_nodes,
        "num_passthrough_nodes": num_passthrough_nodes,
        "num_split_nodes": num_split_nodes,
        "num_observed_nodes": num_observed_nodes,
        "num_inferred_nodes": num_inferred_nodes,
    },
    index=[0],
)
tmpdf

,cloneid,mouseID,V_gene,J_gene,CDR3_len,num_nodes,num_leaves,num_internal_nodes,num_passthrough_nodes,num_split_nodes,num_observed_nodes,num_inferred_nodes
0,m31_IGHV14-3-01_IGHJ1-03_42_1.aln,m31,IGHV14-3-01,IGHJ1-03,42,3,1,1,1,0,2,1


'm31_IGHV14-3-01_IGHJ1-03_42_1.aln'